In [ ]:
%pip install openai pydantic --quiet

# Tool Calling (Function Calling) with OpenAI

Function calling provides a powerful and flexible way for OpenAI models to interface with your code or external services. It allows models to fetch data and take actions by calling functions you define—effectively extending the model’s capabilities. In this notebook, we cover several examples:

- **Get Weather:** Retrieve the current temperature using provided coordinates.
- **Send Email:** Simulate sending an email message.
- **Search Knowledge Base:** Query a knowledge base for information.
- **Web Search:** Use a web search tool to retrieve recent news or other content.

We also explain how to handle function call events when streaming. 

Let's get started!

## Example 1: Get Weather Function

In this example, we define a `get_weather` tool that retrieves the current temperature for provided coordinates. Notice how the tool is defined using a JSON schema in the `tools` list (with strict mode enabled). 

The API call returns a tool call object. To fix the error you saw, we access its attributes (e.g. `.type` and `.arguments`) rather than subscript it like a dictionary.

**Python Code Example:**

In [2]:
from openai import OpenAI
import json
import requests

In [ ]:
MODEL = "gpt-4.1-mini"

In [3]:
# Create an OpenAI client instance
client = OpenAI(
    # Replace with your actual API key or use: api_key=os.environ.get("OPENAI_API_KEY")
    api_key="YOUR_API_KEY_HERE"
)

In [4]:
# Sample implementation for the get_weather function
def get_weather(latitude, longitude):
    response = requests.get(
        f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m"
    )
    data = response.json()
    return data['current']['temperature_2m']

# Define the get_weather tool with strict mode
tools = [{
    "type": "function",
    "name": "get_weather",
    "description": "Get current temperature for provided coordinates in celsius.",
    "parameters": {
        "type": "object",
        "properties": {
            "latitude": {"type": "number", "description": "Latitude of the location."},
            "longitude": {"type": "number", "description": "Longitude of the location."}
        },
        "required": ["latitude", "longitude"],
        "additionalProperties": False
    },
    "strict": True
}]

# Input message asking about the weather in Paris
input_messages = [{"role": "user", "content": "What's the weather like in Paris today?"}]

# Call the API
response = client.responses.create(
    model=MODEL,
    input=input_messages,
    tools=tools
)

# Print the model's output (expected to include a function call)
print("Get Weather Response Output:", response.output)

# If the model calls the function, access its attributes (note the use of dot-access).
if response.output and response.output[0].type == "function_call":
    tool_call = response.output[0]
    args = json.loads(tool_call.arguments)
    weather_result = get_weather(args['latitude'], args['longitude'])
    print(f"Weather Result: {weather_result}°C")

# Add the function call output to the input messages and call the API again.
input_messages.append(tool_call)  # append model's function call message
input_messages.append({                               # append result message
    "type": "function_call_output",
    "call_id": tool_call.call_id,
    "output": str(json.dumps(weather_result) )
})

# Call the API again with the updated input messages
response = client.responses.create(
    model=MODEL,
    input=input_messages,
    tools=tools
)

print("Get Weather Response Output:", response.output_text)

Get Weather Response Output: [ResponseFunctionToolCall(arguments='{"latitude":48.8566,"longitude":2.3522}', call_id='call_KJThMN9cz0pJLesi3fnxjB17', name='get_weather', type='function_call', id='fc_67fff71414b48192b5d5e16832325e0805a286687b3e5c09', status='completed')]
Weather Result: 13.2°C
Get Weather Response Output: The weather in Paris today is approximately 13.2°C. If you need more details such as forecasts or conditions (e.g., sunny, rainy), let me know!


## Example 2: Web Search Tool

Finally, we add a web search tool. This tool lets the model retrieve the latest online information. It can be used to fetch timely data such as news or restaurant recommendations. 

The following example shows how to use the `web_search_preview` tool with the Responses API. Note that you can also customize user location and search context size if needed.

**Python Code Example:**

In [5]:
# Define the web search tool (with default configuration)
tools = [{
    "type": "web_search_preview"
}]

# Input message for web search
input_message = "What was a positive news story from today?"

# Call the API with the web search tool
response = client.responses.create(
    model=MODEL,
    tools=tools,
    input=[{"role": "user", "content": input_message}]
)

print("Web Search Tool Response Output:", response.output_text)

# Optionally, you can force lower latency using the tool_choice parameter or customize user location

# Example with user location and low search context size
tools_with_location = [{
    "type": "web_search_preview",
    "user_location": {
         "type": "approximate",
         "country": "GB",
         "city": "London",
         "region": "London"
    },
    "search_context_size": "low"
}]

response = client.responses.create(
    model=MODEL,
    tools=tools_with_location,
    input=[{"role": "user", "content": "What are the best restaurants around Granary Square?"}]
)

print("Web Search with Location Response Output:", response.output_text)

Web Search Tool Response Output: As of April 16, 2025, one notable positive news story is the Democratic Republic of Congo's (DRC) commitment to establish the world's largest protected tropical reserve, known as the Couloir Vert or 'Green Corridor'. This expansive reserve will cover an area approximately the size of France, aiming to conserve vast tracts of the Congo Basin, including forests and peatlands. The Congo Basin is recognized as the largest tropical carbon sink globally and is home to a diverse array of species, such as the eastern lowland gorillas. This initiative underscores the DRC's dedication to addressing climate change and preserving biodiversity. ([positive.news](https://www.positive.news/society/good-news-stories-from-week-04-of-2025/?utm_source=openai)) 
Web Search with Location Response Output: Granary Square in London's King's Cross area boasts a vibrant dining scene. Here are some top-rated restaurants you might consider:

**[Caravan King's Cross](https://www.goo

---

## Streaming with Tool Calling

In [6]:
tools = [{
    "type": "function",
    "name": "get_weather",
    "description": "Get current temperature for a given location.",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "City and country e.g. Bogotá, Colombia"
            }
        },
        "required": [
            "location"
        ],
        "additionalProperties": False
    }
}]

stream = client.responses.create(
    model=MODEL,
    input=[{"role": "user", "content": "What's the weather like in Paris today?"}],
    tools=tools,
    stream=True
)

for event in stream:
    print(event)

ResponseCreatedEvent(response=Response(id='resp_67fff72f2ac8819282380ffece5c6be00f20e5195cc62e4a', created_at=1744828207.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4.1-2025-04-14', object='response', output=[], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[FunctionTool(name='get_weather', parameters={'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'City and country e.g. Bogotá, Colombia'}}, 'required': ['location'], 'additionalProperties': False}, strict=True, type='function', description='Get current temperature for a given location.')], top_p=1.0, max_output_tokens=None, previous_response_id=None, reasoning=Reasoning(effort=None, generate_summary=None, summary=None), status='in_progress', text=ResponseTextConfig(format=ResponseFormatText(type='text')), truncation='disabled', usage=None, user=None, store=True), type='response.created')
ResponseInProgressEvent(response=Response(id='resp_67fff

## Summary and Best Practices

- **Tool Calling Overview:** Tool calling enables your model to trigger custom functions based on conversation context. Use it to fetch data or perform actions.

- **Defining Functions:** Define your functions in the `tools` parameter using JSON schema. Always set `additionalProperties` to false and enable strict mode for more reliable results.

- **Handling Responses:** When the model calls a function, it returns a function call object (with attributes like `type` and `arguments`). Access attributes with dot notation (e.g. `tool_call.arguments`).

- **Web Search Tool:** You can add web search functionality as another tool. Customize user location and search context size as needed.

This notebook demonstrates several tools and best practices for function calling in OpenAI's Responses API. Use and modify these examples for high-quality integrations in your applications or teaching materials.